In [1]:
import time
import logging
from IPython.display import Audio

from IDRnD.utils import *
from IDRnD.augmentations import *
from IDRnD.dataset import *
from IDRnD.resnet import *
from IDRnD.nasnet_mobile import NASNetAMobile
from IDRnD.focalloss import FocalLoss
from IDRnD.callbacks import *
from IDRnD.pipeline import *
from IDRnD.mobilenet import *

import numpy as np
import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, CyclicLR, ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

from sklearn.model_selection import StratifiedKFold, train_test_split

%reload_ext autoreload
%autoreload 2
%matplotlib inline

seed_everything(0)
logging.basicConfig(level=logging.DEBUG, filename="logs/logs.log",
                    filemode="w+")

In [2]:
post_transform = transforms.Compose([
    librosa.power_to_db,
    PadOrClip(320),
    Normalize_predef(-29.6179, 16.6342),
    ToTensor(),
    #transforms.ToTensor(),
    #lambda x: x.view(-1),
])

In [3]:
X_base, y_base = get_train_data(True)
#common_X, common_y = get_common_voices()
pathes_old_competition, classes_old_competition = get_old_competition_dataset(False)

#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, y_train, X_val, y_val = np.load("IDRnD/data/four_cicle_loop_validation.npy", allow_pickle=True)

#mask = np.load("IDRnD/data/mask.npy")
#X_good, X_bad, y_good, y_bad = X_val[np.invert(mask)], X_val[mask], y_val[np.invert(mask)], y_val[mask]
#X_train_new, y_train_new = np.concatenate((X_train, X_bad)), np.concatenate((y_train, y_bad))
#X_val_new, y_val_new = X_good, y_good

#X, y = np.concatenate((common_X, pathes_old_competition)), np.concatenate((common_y, classes_old_competition))
clusters=pd.read_csv("IDRnD/data/clusters_resnet.csv")["cluster"].values
white_list = np.load("IDRnD/data/white_list.npy")
clusters = clusters[white_list]
X, y = pathes_old_competition, classes_old_competition

In [4]:
!rm -rf experiments
from mag.experiment import Experiment
from IDRnD.dimka.networks.classifiers import TwoDimensionalCNNClassificationModel

with Experiment({
    "data": {
        "_input_dim": 64,
        "_kfold_seed": 42,
        "_n_folds": 5,
        "_train_data_dir": "data/Training_Data/",
        "_train_df": "data/train_df.csv",
        "features": "mel_1024_512_64",
        "max_audio_length": 3,
        "p_aug": 0.3,
        "p_mixup": 0.0
    },
    "label": "2d_cnn",
    "network": {
        "aggregation_type": "max",
        "conv_base_depth": 32,
        "growth_rate": 1.3,
        "num_conv_blocks": 5,
        "output_dropout": 0.5,
        "start_deep_supervision_on": 2
    },
    "train": {
        "_save_every": 5,
        "accumulation_steps": 1,
        "batch_size": 50,
        "epochs": 7,
        "learning_rate": 0.001,
        "optimizer": "adam",
        "scheduler": "1cycle_0.0001_0.005",
        "switch_off_augmentations_on": 6,
        "weight_decay": 0.0
    }
}) as experiment:
    device = "cuda"
    config = experiment.config
    model = TwoDimensionalCNNClassificationModel(experiment, device=device)


In [ ]:
batch_size = 300

for i in range(5):
    upper, down = (i+1)*20, 20*i
    down_ = clusters >= down
    upper_ = clusters < upper
    mask = down_ & upper_
    X_train, y_train, X_val, y_val = X_base[~mask], y_base[~mask], X_base[mask], y_base[mask]
    
    state = torch.load('models/dimka_common_and_old.pt')
    model.load_state_dict(state)
    
    train_dataset = SimpleMelDataset(X_train, y_train, "../data/files/raw_mels/", post_transform)
    valid_dataset = SimpleMelDataset(X_val, y_val, "../data/files/raw_mels/", post_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size*3, num_workers=16, shuffle=False)
    model_dst = torch.nn.DataParallel(model, device_ids=[0, 1]).cuda()

    criterion = nn.BCELoss().cuda()
    #criterion = FocalLoss(gamma=2, reduce='mean').cuda()
    optimizer = Adam(params=model.parameters(), lr=1e-4)

    tb_logger = TensorBoardCallback(compute_eer)
    last = SaveLastEpoch("models/temp.pt")
    acumulator = AccumulateGradient([30, 50, 70, 100])
    best = SaveBestEpoch(f"models/dimka_model_from_pretrain_fold_{i}.pt", compute_eer)
    scheduler = ReduceLROnPlateau(optimizer, patience=1, verbose=True)
    scheduler_call = EpochScheduler(scheduler)
    hm = Train(callbacks=[tb_logger, acumulator, best, scheduler_call, last])
    hm.fit(train_loader, valid_loader, model_dst, criterion, optimizer, epoches=70)
    print("Done!")

Epoch    17: reducing learning rate of group 0 to 1.0000e-05.
Epoch    30: reducing learning rate of group 0 to 1.0000e-06.
Epoch    34: reducing learning rate of group 0 to 1.0000e-07.
Epoch    36: reducing learning rate of group 0 to 1.0000e-08.
Done!
Epoch     9: reducing learning rate of group 0 to 1.0000e-05.
Epoch    20: reducing learning rate of group 0 to 1.0000e-06.
Epoch    25: reducing learning rate of group 0 to 1.0000e-07.
Epoch    29: reducing learning rate of group 0 to 1.0000e-08.


In [10]:
true, preds = None, None
batch_size = 300
for i in range(5):
    upper, down = (i+1)*20, 20*i
    down_ = clusters >= down
    upper_ = clusters < upper
    mask = down_ & upper_
    X_train, y_train, X_val, y_val = X_base[~mask], y_base[~mask], X_base[mask], y_base[mask]
    
    state = torch.load(f'models/dimka_model_from_pretrain_fold_{i}.pt')
    model.load_state_dict(state)

    valid_dataset = SimpleMelDataset(X_val, y_val, "../data/files/raw_mels/", post_transform)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size*3, num_workers=16, shuffle=False)
    model_dst = torch.nn.DataParallel(model, device_ids=[0, 1]).cuda()
    pred = hm.predict_on_test(valid_loader, model_dst).values
    
    if true is None:
        true, preds = y_val, pred
    else:
        true, preds = np.concatenate((true, y_val)), np.concatenate((preds, pred))

In [5]:
##train
batch_size = 300

#train_dataset = BaseDataset(pathes_old_competition[:-1000], classes_old_competition[:-1000], post_transform)# pathes_old_competition
#valid_dataset = BaseDataset(pathes_old_competition[-1000:], classes_old_competition[-1000:], post_transform)# pathes_old_competition
train_dataset = SimpleMelDataset(X, y, "../data/files/raw_mels/", post_transform)
valid_dataset = SimpleMelDataset(X_base, y_base, "../data/files/raw_mels/", post_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size*3, num_workers=16, shuffle=False)

#model = resnet34(num_classes=1).cuda()
#model = mobilenetv2(num_classes=1, input_size=128).cuda()
#state = torch.load('models/resnet_34_four_cicle_validation.pt')
#state.pop("conv1.weight")
#state.pop("fc.weight")
#state.pop("fc.bias")
#model.load_state_dict(state, strict=False)
model_dst = torch.nn.DataParallel(model, device_ids=[0, 1]).cuda()

criterion = nn.BCELoss().cuda()
#criterion = FocalLoss(gamma=2, reduce='mean').cuda()
optimizer = Adam(params=model.parameters(), lr=1e-4)

In [6]:
tb_logger = TensorBoardCallback(compute_eer)
#saver = SaveEveryEpoch("models/resnet_34_common_voice.pt")
last = SaveLastEpoch("models/temp.pt")
acumulator = AccumulateGradient([30, 50, 70, 100])
best = SaveBestEpoch("models/dimka_common_and_old.pt", compute_eer)
scheduler = ReduceLROnPlateau(optimizer, patience=1, verbose=True)
scheduler_call = EpochScheduler(scheduler)
hm = Train(callbacks=[tb_logger, acumulator, best, scheduler_call, last])

hm.fit(train_loader, valid_loader, model_dst, criterion, optimizer, epoches=150)

Epoch    17: reducing learning rate of group 0 to 1.0000e-05.
Epoch    35: reducing learning rate of group 0 to 1.0000e-06.
Epoch    37: reducing learning rate of group 0 to 1.0000e-07.
Epoch    42: reducing learning rate of group 0 to 1.0000e-08.


KeyboardInterrupt: 